# Матренин Василий Николаевич. Б01-008
# Лабораторная работа \#2


In [2]:
import numpy as np

def if_LU_decomposable(A) :
    for i in range(0, A.shape[0]) :
        M = np.delete(np.delete(A, i, 0), i, 1)
        if np.linalg.det(A) == 0 :
            return False
        
    return True

def LU_decompose(A) :
    N = A.shape[0]
    L = np.eye(N)
    U = np.zeros((N, N))
    
    for i in range(0, N) :
        for j in range(0, N) :
            if i <= j :
                U[i][j] = A[i][j] - np.sum([L[i][k] * U[k][j] for k in range(0, i)])
            else :
                L[i][j] = (A[i][j] - np.sum([L[i][k] * U[k][j] for k in range(0, j)])) / U[j][j]
    
    return [L, U]

def solve_U_mat(U, b) :
    N = U.shape[0]
    x = np.zeros(N)
    x[N-1] = b[N-1] / U[N-1][N-1]
    
    for i in range(1, N) :
        i = N-i-1
        x[i] = (b[i] - np.sum([x[k] * U[i][k] for k in range(i+1, N)])) / U[i][i]

    return x
    
def solve_L_mat(L, b) :
    N = L.shape[0]
    x = np.zeros(N)
    x[0] = b[0] / L[0][0]
    
    for i in range(1, N) :
        x[i] = (b[i] - np.sum([x[k] * L[i][k] for k in range(0, i)])) / L[i][i]

    return x

def solve_gauss(A, b) :
    N = A.shape[0]
    Ab = np.insert(A, N, b, axis=1)
    
    # Forward Gauss pass
    for k in range(1, N) :
        for j in range(k, N) :
            factor = Ab[j][k-1] / Ab[k-1][k-1]

            for i in range(0, N+1) :
                Ab[j][i] = Ab[j][i] - factor * Ab[k-1][i]

    x = np.zeros(N)

    # Backward Gauss pass
    for i in reversed(range(0, N)) :
        x[i] = Ab[i][N] / Ab[i][i]
        for c in reversed(range(i+1, N)) :
            x[i] = x[i] - Ab[i][c] * x[c] / Ab[i][i]
            
    return x

def norm_1(vec) :
    return np.amax(abs(vec))


In [3]:
import numpy as np

def LDU_sum_decompose(A) :
    N = A.shape[0]
    L = np.zeros((N, N))
    D = np.zeros((N, N))
    U = np.zeros((N, N))
    
    for i in range(0, N) :
        for j in range(0, i) :
            L[i][j] = A[i][j]
        
        D[i][i] = A[i][i]

        for j in range(i+1, N) :
            U[i][j] = A[i][j]
            
    return [L, D, U]

def calculate_relaxation_coeffs(A, f, omega) :
    [L, D, U] = LDU_sum_decompose(A)

    # x^(k+1) = factor * x^(k) + addition
    factor = -np.linalg.inv(D + omega*L).dot((omega-1)*D + omega*U)
    addition = omega * np.linalg.inv(D + omega*L).dot(f)
    
    return [factor, addition]

def solve_relaxation(factor, addition, x0, epsilon) :
    x = x0
    x_prev = x0 + epsilon * 2
    
    while norm_1(x - x_prev) >= epsilon :
        x_prev = x
        x = factor.dot(x) + addition
    
    return x


In [4]:
import numpy as np

def process_equation(A, b, omega, x0, epsilon) :
    N = A.shape[0]
    
    print("A:\n", A)
    print("b:\n", b)

    if if_LU_decomposable(A) :
        [L, U] = LU_decompose(A)
        x_l = solve_L_mat(L, b)
        x_ref = solve_U_mat(U, x_l)
    else :
        x_ref = solve_gauss(A, b)

    [factor, addition] = calculate_relaxation_coeffs(A, b, omega)
    x_approx = solve_relaxation(factor, addition, x0, epsilon)

    print("reference x:\n", x_ref)
    print("approximate x:\n", x_approx)

    print("error:\n", norm_1(x_ref - x_approx))


In [5]:
# 2.10.5.а

import numpy as np

np.set_printoptions(precision=4)

N = 99

A = np.eye(N+1, k=1) + np.eye(N+1, k=-1) + np.eye(N+1)*10
A[N] = np.ones(N+1)

b = np.array(range(1, N+2));

process_equation(A, b, omega = 1.2, x0 = np.ones(N+1), epsilon = 1e-10)


A:
 [[10.  1.  0. ...  0.  0.  0.]
 [ 1. 10.  1. ...  0.  0.  0.]
 [ 0.  1. 10. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 10.  1.  0.]
 [ 0.  0.  0. ...  1. 10.  1.]
 [ 1.  1.  1. ...  1.  1.  1.]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [ 8.3333e-02  1.6667e-01  2.5000e-01  3.3333e-01  4.1667e-01  5.0000e-01
  5.8333e-01  6.6667e-01  7.5000e-01  8.3333e-01  9.1667e-01  1.0000e+00
  1.0833e+00  1.1667e+00  1.2500e+00  1.3333e+00  1.4167e+00  1.5000e+00
  1.5833e+00  1.6667e+00  1.7500e+00  1.8333e+00  1.9167e+00  2.0000e+00
  2.0833e+00  2.1667e+00  2.2500e+00  2.3333e+00  2.4167e+00  2.5000e+00
  2.5833e+00  

In [6]:
# 2.10.5.б

import numpy as np

np.set_printoptions(precision=4)

N = 9

A = np.eye(N+1, k=1) + np.eye(N+1, k=-1) - 2*np.eye(N+1)
A[N] = 2*np.ones(N+1)
A[0][0] = 1
A[0][1] = 0
A[N][0] = 1
A[N][N] = 1

b = 2 / np.array(range(1, N+2)) ** 2;
b[0] = 1
b[N] = -N / 3

process_equation(A, b, omega = 1, x0 = -np.ones(N+1), epsilon = 1e-10)


A:
 [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -2.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -2.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -2.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -2.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1. -2.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1. -2.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -2.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -2.  1.]
 [ 1.  2.  2.  2.  2.  2.  2.  2.  2.  1.]]
b:
 [ 1.      0.5     0.2222  0.125   0.08    0.0556  0.0408  0.0312  0.0247
 -3.    ]
reference x:
 [ 1.      0.1136 -0.2727 -0.4368 -0.476  -0.4351 -0.3387 -0.2015 -0.033
  0.1602]
approximate x:
 [ 1.      0.1136 -0.2727 -0.4368 -0.476  -0.4351 -0.3387 -0.2015 -0.033
  0.1602]
error:
 1.574401164994299e-10


In [7]:
# 2.10.5.в

import numpy as np

np.set_printoptions(precision=4)

N = 19

A = np.eye(N+1, k=1) + np.eye(N+1, k=-1) - 2*np.eye(N+1)
A[N] = 2*np.ones(N+1)
A[0][0] = 1
A[0][1] = 0
A[N][0] = 1
A[N][N] = 1

b = 2 / np.array(range(1, N+2)) ** 2;
b[0] = 1
b[N] = -N / 3

process_equation(A, b, omega = 1, x0 = -np.ones(N+1), epsilon = 1e-10)


A:
 [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -2.  1.  0.  0.

In [8]:
# 2.10.5.г

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = np.eye(N, k=1) + np.eye(N, k=-1) + 10*np.eye(N)
A[0] = np.ones(N)
A[N-1][N-1] = 1

b = np.flip(np.array(range(1, N+1)))

process_equation(A, b, omega = 1, x0 = -np.ones(N), epsilon = 1e-10)


A:
 [[ 1.  1.  1. ...  1.  1.  1.]
 [ 1. 10.  1. ...  0.  0.  0.]
 [ 0.  1. 10. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 10.  1.  0.]
 [ 0.  0.  0. ...  1. 10.  1.]
 [ 0.  0.  0. ...  0.  1.  1.]]
b:
 [100  99  98  97  96  95  94  93  92  91  90  89  88  87  86  85  84  83
  82  81  80  79  78  77  76  75  74  73  72  71  70  69  68  67  66  65
  64  63  62  61  60  59  58  57  56  55  54  53  52  51  50  49  48  47
  46  45  44  43  42  41  40  39  38  37  36  35  34  33  32  31  30  29
  28  27  26  25  24  23  22  21  20  19  18  17  16  15  14  13  12  11
  10   9   8   7   6   5   4   3   2   1]
reference x:
 [-3.4575e+02  4.4019e+01  4.5532e+00  8.4484e+00  7.9631e+00  7.9204e+00
  7.8330e+00  7.7500e+00  7.6667e+00  7.5833e+00  7.5000e+00  7.4167e+00
  7.3333e+00  7.2500e+00  7.1667e+00  7.0833e+00  7.0000e+00  6.9167e+00
  6.8333e+00  6.7500e+00  6.6667e+00  6.5833e+00  6.5000e+00  6.4167e+00
  6.3333e+00  6.2500e+00  6.1667e+00  6.0833e+00  6.0000e+00  5.9167e+00
  5.8333e+00  

In [9]:
# 2.10.5.д

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = 10*np.eye(N)
for i in range(1, 5) :
    A = A + np.eye(N, k=i) + np.eye(N, k=-i)

b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[10.  1.  1. ...  0.  0.  0.]
 [ 1. 10.  1. ...  0.  0.  0.]
 [ 1.  1. 10. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 10.  1.  1.]
 [ 0.  0.  0. ...  1. 10.  1.]
 [ 0.  0.  0. ...  1.  1. 10.]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [0.0229 0.0969 0.1649 0.2267 0.2823 0.3342 0.3882 0.4435 0.4996 0.5556
 0.6113 0.6668 0.7222 0.7777 0.8333 0.8889 0.9445 1.     1.0556 1.1111
 1.1667 1.2222 1.2778 1.3333 1.3889 1.4444 1.5    1.5556 1.6111 1.6667
 1.7222 1.7778 1.8333 1.8889 1.9444 2.     2.0556 2.1111 2.1667 2.2222
 2.2778 2.3333 2.3889 2.4444 2.5    2.5556 2.6111 2.6667 2.7222 2.7778
 2.8333 2.8889 2.9444 3.

In [10]:
# 2.10.5.е

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = 10*np.eye(N)
for i in range(0, N) :
    i_str_value = 1/(i+1)
    if i != N-1 :
        A[i][i+1] = i_str_value
    for j in range(0, i) :
        A[i][j] = i_str_value


b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[10.      1.      0.     ...  0.      0.      0.    ]
 [ 0.5    10.      0.5    ...  0.      0.      0.    ]
 [ 0.3333  0.3333 10.     ...  0.      0.      0.    ]
 ...
 [ 0.0102  0.0102  0.0102 ... 10.      0.0102  0.    ]
 [ 0.0101  0.0101  0.0101 ...  0.0101 10.      0.0101]
 [ 0.01    0.01    0.01   ...  0.01    0.01   10.    ]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [0.0818 0.182  0.2787 0.3747 0.4703 0.5659 0.6613 0.7567 0.852  0.9473
 1.0426 1.1379 1.2332 1.3285 1.4238 1.5191 1.6143 1.7096 1.8048 1.9001
 1.9954 2.0906 2.1859 2.2811 2.3764 2.4716 2.5669 2.6621 2.7574 2.8526
 2.9478 3.0431 3.1383 

In [11]:
# 2.10.5.ж

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = 20*np.eye(N)
for i in range(1, 5) :
    A = A + np.eye(N, k=i) + np.eye(N, k=-i)


b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[20.  1.  1. ...  0.  0.  0.]
 [ 1. 20.  1. ...  0.  0.  0.]
 [ 1.  1. 20. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 20.  1.  1.]
 [ 0.  0.  0. ...  1. 20.  1.]
 [ 0.  0.  0. ...  1.  1. 20.]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [0.0252 0.0666 0.106  0.1436 0.1794 0.2145 0.25   0.2856 0.3214 0.3571
 0.3929 0.4286 0.4643 0.5    0.5357 0.5714 0.6071 0.6429 0.6786 0.7143
 0.75   0.7857 0.8214 0.8571 0.8929 0.9286 0.9643 1.     1.0357 1.0714
 1.1071 1.1429 1.1786 1.2143 1.25   1.2857 1.3214 1.3571 1.3929 1.4286
 1.4643 1.5    1.5357 1.5714 1.6071 1.6429 1.6786 1.7143 1.75   1.7857
 1.8214 1.8571 1.8929 1.

In [12]:
# 2.10.5.з

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = 10*np.eye(N)
for i in range(0, N) :
    i_str_value = 1/(i+1)
    if i != N-1 :
        A[i][i+1] = i_str_value
    for j in range(0, i) :
        A[i][j] = i_str_value


b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[10.      1.      0.     ...  0.      0.      0.    ]
 [ 0.5    10.      0.5    ...  0.      0.      0.    ]
 [ 0.3333  0.3333 10.     ...  0.      0.      0.    ]
 ...
 [ 0.0102  0.0102  0.0102 ... 10.      0.0102  0.    ]
 [ 0.0101  0.0101  0.0101 ...  0.0101 10.      0.0101]
 [ 0.01    0.01    0.01   ...  0.01    0.01   10.    ]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [0.0818 0.182  0.2787 0.3747 0.4703 0.5659 0.6613 0.7567 0.852  0.9473
 1.0426 1.1379 1.2332 1.3285 1.4238 1.5191 1.6143 1.7096 1.8048 1.9001
 1.9954 2.0906 2.1859 2.2811 2.3764 2.4716 2.5669 2.6621 2.7574 2.8526
 2.9478 3.0431 3.1383 

In [13]:
# 2.10.5.и

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = 100*np.eye(N)
for i in range(0, N) :
    if i != N-1 :
        A[i][i+1] = (i+1)/(i+2)
    for j in range(0, i) :
        A[i][j] = (i+1)/(j+1)


b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[100.       0.5      0.     ...   0.       0.       0.    ]
 [  2.     100.       0.6667 ...   0.       0.       0.    ]
 [  3.       1.5    100.     ...   0.       0.       0.    ]
 ...
 [ 98.      49.      32.6667 ... 100.       0.9899   0.    ]
 [ 99.      49.5     33.     ...   1.0102 100.       0.99  ]
 [100.      50.      33.3333 ...   1.0204   1.0101 100.    ]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [0.0099 0.0196 0.0291 0.0384 0.0476 0.0565 0.0653 0.0739 0.0823 0.0905
 0.0986 0.1065 0.1142 0.1218 0.1292 0.1365 0.1436 0.1505 0.1573 0.1639
 0.1704 0.1768 0.183  0.189  0.195  0.2007 0.2064 0.2119 

In [14]:
# 2.10.5.к

import numpy as np

np.set_printoptions(precision=4)

N = 10

A = np.eye(N)
for i in range(0, N) :
    for j in range(0, N) :
        if j != i :
            A[i][j] = 1/(i+j+2)


b = 1 / np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[1.     0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909]
 [0.3333 1.     0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833]
 [0.25   0.2    1.     0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769]
 [0.2    0.1667 0.1429 1.     0.1111 0.1    0.0909 0.0833 0.0769 0.0714]
 [0.1667 0.1429 0.125  0.1111 1.     0.0909 0.0833 0.0769 0.0714 0.0667]
 [0.1429 0.125  0.1111 0.1    0.0909 1.     0.0769 0.0714 0.0667 0.0625]
 [0.125  0.1111 0.1    0.0909 0.0833 0.0769 1.     0.0667 0.0625 0.0588]
 [0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667 1.     0.0588 0.0556]
 [0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588 1.     0.0526]
 [0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588 0.0556 0.0526 1.    ]]
b:
 [1.     0.5    0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1   ]
reference x:
 [ 9.1908e-01  1.7554e-01  6.3935e-02  2.7275e-02  1.1423e-02  3.5108e-03
 -7.8996e-04 -3.2508e-03 -4.6979e-03 -5.5537e-03]
approximate x:
 [ 9.1908e-01  1.7554e-01  6.3935e-02

In [15]:
# 2.10.5.л

import numpy as np

np.set_printoptions(precision=4)

N = 20

A = 10*np.eye(N)
for i in range(0, N) :
    for j in range(0, N) :
        if j != i :
            A[i][j] = 1/(i+j+2)


b = 1 / np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[10.      0.3333  0.25    0.2     0.1667  0.1429  0.125   0.1111  0.1
   0.0909  0.0833  0.0769  0.0714  0.0667  0.0625  0.0588  0.0556  0.0526
   0.05    0.0476]
 [ 0.3333 10.      0.2     0.1667  0.1429  0.125   0.1111  0.1     0.0909
   0.0833  0.0769  0.0714  0.0667  0.0625  0.0588  0.0556  0.0526  0.05
   0.0476  0.0455]
 [ 0.25    0.2    10.      0.1429  0.125   0.1111  0.1     0.0909  0.0833
   0.0769  0.0714  0.0667  0.0625  0.0588  0.0556  0.0526  0.05    0.0476
   0.0455  0.0435]
 [ 0.2     0.1667  0.1429 10.      0.1111  0.1     0.0909  0.0833  0.0769
   0.0714  0.0667  0.0625  0.0588  0.0556  0.0526  0.05    0.0476  0.0455
   0.0435  0.0417]
 [ 0.1667  0.1429  0.125   0.1111 10.      0.0909  0.0833  0.0769  0.0714
   0.0667  0.0625  0.0588  0.0556  0.0526  0.05    0.0476  0.0455  0.0435
   0.0417  0.04  ]
 [ 0.1429  0.125   0.1111  0.1     0.0909 10.      0.0769  0.0714  0.0667
   0.0625  0.0588  0.0556  0.0526  0.05    0.0476  0.0455  0.0435  0.0417
   0.04    0.0385]

In [16]:
# 2.10.5.м

import numpy as np

np.set_printoptions(precision=4)

N = 20

A = 10*np.eye(N) + np.eye(N, k=1) + np.eye(N, k=-1) + 0.1*(np.eye(N, k=2) + np.eye(N, k=-2))

b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[10.   1.   0.1  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 1.  10.   1.   0.1  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.1  1.  10.   1.   0.1  0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.1  1.  10.   1.   0.1  0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.1  1.  10.   1.   0.1  0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.1  1.  10.   1.   0.1  0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.1  1.  10.   1.   0.1  0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.1  1.  10.   1.   0.1  0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.1  1.  10.   1.   0.1  0.   0.   0.
   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.1  1.  10.   1.   0.

In [17]:
# 2.10.5.н

import numpy as np

np.set_printoptions(precision=4)

N = 20

A = 10*np.eye(N) + np.eye(N, k=1) + np.eye(N, k=-1)

b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1. 10.  1.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. 10.  1.  0.  0.

In [18]:
# 2.10.5.о

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = np.eye(N, k=-1) + 10*np.eye(N) + np.eye(N, k=1) + 0.1*np.eye(N, k=2)

b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[10.   1.   0.1 ...  0.   0.   0. ]
 [ 1.  10.   1.  ...  0.   0.   0. ]
 [ 0.   1.  10.  ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ... 10.   1.   0.1]
 [ 0.   0.   0.  ...  1.  10.   1. ]
 [ 0.   0.   0.  ...  0.   1.  10. ]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [0.0811 0.1639 0.2466 0.3292 0.4119 0.4945 0.5771 0.6598 0.7424 0.8251
 0.9077 0.9904 1.073  1.1557 1.2383 1.3209 1.4036 1.4862 1.5689 1.6515
 1.7342 1.8168 1.8995 1.9821 2.0647 2.1474 2.23   2.3127 2.3953 2.478
 2.5606 2.6433 2.7259 2.8086 2.8912 2.9738 3.0565 3.1391 3.2218 3.3044
 3.3871 3.4697 3.5524 3.635  3.7176 3.8003 3.8829 3.9656 4.0

In [19]:
# 2.10.5.п

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = np.eye(N, k=-1) + np.eye(N, k=1)
for i in range(0, N) :
    A[i][i] = 11 + i
A[N-1] = 2
A[N-1][0] = 1
A[N-1][N-1] = 1

b = np.array(range(1, N+1)) / (N-1)

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[ 11.   1.   0. ...   0.   0.   0.]
 [  1.  12.   1. ...   0.   0.   0.]
 [  0.   1.  13. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 108.   1.   0.]
 [  0.   0.   0. ...   1. 109.   1.]
 [  1.   2.   2. ...   2.   2.   1.]]
b:
 [0.0101 0.0202 0.0303 0.0404 0.0505 0.0606 0.0707 0.0808 0.0909 0.101
 0.1111 0.1212 0.1313 0.1414 0.1515 0.1616 0.1717 0.1818 0.1919 0.202
 0.2121 0.2222 0.2323 0.2424 0.2525 0.2626 0.2727 0.2828 0.2929 0.303
 0.3131 0.3232 0.3333 0.3434 0.3535 0.3636 0.3737 0.3838 0.3939 0.404
 0.4141 0.4242 0.4343 0.4444 0.4545 0.4646 0.4747 0.4848 0.4949 0.5051
 0.5152 0.5253 0.5354 0.5455 0.5556 0.5657 0.5758 0.5859 0.596  0.6061
 0.6162 0.6263 0.6364 0.6465 0.6566 0.6667 0.6768 0.6869 0.697  0.7071
 0.7172 0.7273 0.7374 0.7475 0.7576 0.7677 0.7778 0.7879 0.798  0.8081
 0.8182 0.8283 0.8384 0.8485 0.8586 0.8687 0.8788 0.8889 0.899  0.9091
 0.9192 0.9293 0.9394 0.9495 0.9596 0.9697 0.9798 0.9899 1.     1.0101]
reference x:
 [ 0.0008  0.0014  0.002   0.0025  0.003   0

In [20]:
# 2.10.5.р

import numpy as np

np.set_printoptions(precision=4)

N = 50

A = np.eye(N, k=-1) + 5*np.eye(N) + np.eye(N, k=1)
A[N-1] = 1

b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[5. 1. 0. ... 0. 0. 0.]
 [1. 5. 1. ... 0. 0. 0.]
 [0. 1. 5. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 1. 0.]
 [0. 0. 0. ... 1. 5. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
b:
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50]
reference x:
 [ 1.4286e-01  2.8571e-01  4.2857e-01  5.7143e-01  7.1429e-01  8.5714e-01
  1.0000e+00  1.1429e+00  1.2857e+00  1.4286e+00  1.5714e+00  1.7143e+00
  1.8571e+00  2.0000e+00  2.1429e+00  2.2857e+00  2.4286e+00  2.5714e+00
  2.7143e+00  2.8571e+00  3.0000e+00  3.1429e+00  3.2857e+00  3.4286e+00
  3.5714e+00  3.7143e+00  3.8571e+00  4.0000e+00  4.1429e+00  4.2857e+00
  4.4286e+00  4.5714e+00  4.7143e+00  4.8571e+00  5.0000e+00  5.1429e+00
  5.2857e+00  5.4286e+00  5.5714e+00  5.7143e+00  5.8573e+00  5.9994e+00
  6.1456e+00  6.2725e+00  6.4918e+00  6.2683e+00  8.1664e+00 -1.0050e-01
  4.0336e+01 -1.5258e+02]
approximate x:
 [ 1.4286e-01  2.8571e-01  4.2857e-01  5.71

In [21]:
# 2.10.5.с

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = 30*np.eye(N)
for i in range(1, 5) :
    A = A + np.eye(N, k=i) + np.eye(N, k=-i)

b = np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[30.  1.  1. ...  0.  0.  0.]
 [ 1. 30.  1. ...  0.  0.  0.]
 [ 1.  1. 30. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 30.  1.  1.]
 [ 0.  0.  0. ...  1. 30.  1.]
 [ 0.  0.  0. ...  1.  1. 30.]]
b:
 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]
reference x:
 [0.0211 0.0502 0.0783 0.1055 0.1318 0.158  0.1842 0.2105 0.2368 0.2632
 0.2895 0.3158 0.3421 0.3684 0.3947 0.4211 0.4474 0.4737 0.5    0.5263
 0.5526 0.5789 0.6053 0.6316 0.6579 0.6842 0.7105 0.7368 0.7632 0.7895
 0.8158 0.8421 0.8684 0.8947 0.9211 0.9474 0.9737 1.     1.0263 1.0526
 1.0789 1.1053 1.1316 1.1579 1.1842 1.2105 1.2368 1.2632 1.2895 1.3158
 1.3421 1.3684 1.3947 1.

In [22]:
# 2.10.5.т

import numpy as np

np.set_printoptions(precision=4)

N = 100

A = 10*np.eye(N)
for i in range(0, N) :
    if i != N-1 :
        A[i][i+1] = (i+2)/(2*i+3)
    for j in range(0, i) :
        A[i][j] = (j+1)/(i+j+2)


b = np.array(range(1, N+1)) + 2

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[1.0000e+01 6.6667e-01 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.3333e-01 1.0000e+01 6.0000e-01 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.5000e-01 4.0000e-01 1.0000e+01 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [1.0101e-02 2.0000e-02 2.9703e-02 ... 1.0000e+01 5.0254e-01 0.0000e+00]
 [1.0000e-02 1.9802e-02 2.9412e-02 ... 4.9746e-01 1.0000e+01 5.0251e-01]
 [9.9010e-03 1.9608e-02 2.9126e-02 ... 4.9495e-01 4.9749e-01 1.0000e+01]]
b:
 [  3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20
  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102]
reference x:
 [0.2757 0.3639 0.4483 0.5294 0.6074 0.6826 0.755  0.8247 0.8919 0.9567
 1.0192 1.0794 1.1375 1.1936 1.2476 1.2998 1.3502 1.3987

In [23]:
# 2.10.5.у

import numpy as np

np.set_printoptions(precision=4)

N = 12

A = np.eye(N)
for i in range(0, N) :
    for j in range(0, N) :
        if i != j :
            A[i][j] = 1/((i+1)**2+j+1)


b = 1 / np.array(range(1, N+1))

process_equation(A, b, omega = 1.2, x0 = np.ones(N), epsilon = 1e-10)


A:
 [[1.     0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909
  0.0833 0.0769]
 [0.2    1.     0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714
  0.0667 0.0625]
 [0.1    0.0909 1.     0.0769 0.0714 0.0667 0.0625 0.0588 0.0556 0.0526
  0.05   0.0476]
 [0.0588 0.0556 0.0526 1.     0.0476 0.0455 0.0435 0.0417 0.04   0.0385
  0.037  0.0357]
 [0.0385 0.037  0.0357 0.0345 1.     0.0323 0.0312 0.0303 0.0294 0.0286
  0.0278 0.027 ]
 [0.027  0.0263 0.0256 0.025  0.0244 1.     0.0233 0.0227 0.0222 0.0217
  0.0213 0.0208]
 [0.02   0.0196 0.0192 0.0189 0.0185 0.0182 1.     0.0175 0.0172 0.0169
  0.0167 0.0164]
 [0.0154 0.0152 0.0149 0.0147 0.0145 0.0143 0.0141 1.     0.0137 0.0135
  0.0133 0.0132]
 [0.0122 0.012  0.0119 0.0118 0.0116 0.0115 0.0114 0.0112 1.     0.011
  0.0109 0.0108]
 [0.0099 0.0098 0.0097 0.0096 0.0095 0.0094 0.0093 0.0093 0.0092 1.
  0.009  0.0089]
 [0.0082 0.0081 0.0081 0.008  0.0079 0.0079 0.0078 0.0078 0.0077 0.0076
  1.     0.0075]
 [0.0069 0.0068 0.0068